# Examples of alignments with different score parameters
Let\`s try to align some reads with fragments of V-segments on V-segment with different parameters

In [212]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio import AlignIO
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Align.Applications import MuscleCommandline
import random
from auxiliary import *

In [26]:
# Train example

# Alignment module from Biopython is similar to EMBOSS
# pairwise alignment return list of alignments, each of them contains
# seq1, seq2, score, start of alignment (first nongap), end of alignment (last nongap)

templ = 'ATATATTTGTGGG'
a = 'GTGG'
pairwise2.align.localxx(templ, a)

[('ATATATTTGTGGG', '--------GT-GG', 4.0, 8, 13),
 ('ATATATTTGTGGG', '--------GTG-G', 4.0, 8, 13),
 ('ATATATTTGTGGG', '--------GTGG-', 4.0, 8, 12)]

## Different alignments of reads on 1 allele IgV
Ok, how reads can align on allele? There are several variants

In [127]:
# Example allele of IgH v segment with heptamer, spacer and nonamer
allele = 'CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG-----------------------ACACAAACG'

# Load few reads
reads = list(SeqIO.parse('some_reads', 'fasta'))[:10]

In [131]:
# Simple local alignment
for read in reads:
    for x in pairwise2.align.localxx(allele, read.seq)[0]:
        print(x)
    print()

CTGGGCCTGGACCC-AGCAGCCCTCTGGGAAGGCGCTGGGGC-A-CC-TCAGCTCCAGGGGCAGCACACACTTCAGCCCAG--CCTTTCTGGGC-C---AACTCTCC-AT-CTG--T-A---GAGACAC--A----TC-C---AAGGCCCAGTTAT-CCCTGCAGCTGAG--CTCCGTGA-T--GGCCA---AGGGC--AGGGCC---GC----AC-----A---TTCCC-G--T-GGGACAC--AGCG--------------------------AC-----AC-----A-----A-----------AC--G
CT----C---ACCCTA--A-CCCTC-----A--C-C-----CTAACCCTCA-C-CC----G-A--AC-C-C-TCA-CCC-GAACC---CT---CACCCGAA--C-CCGA-AC--CCTCACCCGA-AC-CCTAACCCTCACCCGAA--CCC---TA-ACCC-G--GCT-AGGGCT--G-G-GTTAGG---GTTAGGG-TTAGGG--TTAG-GGTTA-GGGTTAGGGTT---AGGGTTGGG----GTAG-G-----------------------GGTA-GGGGTA-GGGGTAGGGGTAGGGGTGGGGGTA-GGG
114.0
0
312

CT--GGGCCT--GGACCCAGC--AGCCC--TCT-GGG--A-----AGGCGCT--GGG--GC----ACC---TC-AGC--TCC-AGGG---GC----AGC----A-----C-----A-----CA-----C---TTCAG----C-----C-----C-----AGCC--TT----TCT-GGG---CC--AACTC-T--CCATCTGTAGAGACACA-T--CC--AAGGCCCAGTTAT-CCCTGCAG-C--TGAG-CTCCGTGATGG-CC-AAGGGCAGGGCCGCA--C--ATT--CCCGTGGGACAC--AG--C--G-----------------------A-C---AC----AAAC-G-----
GT

Jupyter cell is not the best way to illustrate alignment, yet you can see that we have many gaps in both template and read. We don\`t want such behaviour, cause our template is a continious fragment with defined length. To make it better we can use different penalty for gap.

In [134]:
# With small equal penalty for gap opening and extension
for read in reads:
    for x in pairwise2.align.localxs(allele, read.seq, -1, -1)[0]:
        print(x)
    print()

------------CTGGGCCTGGACC-CAGCAG--CCCTCTGG-GAAGGCGCTGGGGCACCTCAGCTC-CAGGGGCAGCACACACTTCAGCCCAGCCTTT-CTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAG-TTATCCCTGCAGCTGAGCTCCGTG-ATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG-----------------------ACACAAACG
CTCACCCTAACCCTCACCCTAACCCTCACCCGAACCCTCACCCGAACCCTCACCCGAACCCGAACCCTCACCCGAACCCTAACCCTCACCCGAACCCTAACCCGGCTAGGGCT--GGGT-TAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTGGGGTAGGGGTAGGGGTAGGGGTAGGGGTAGGGGTGGG-GGTAGGG--------------------------------
79.0
12
209

------------------------------------------------------------CTGGGCCTGGACCCAGC---AGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTG---GGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGT--TATCCCTGCAGCTGAGC-TCCGTGATGGCCAAGGGCAGGGCCG-CAC-ATTCCCGTGGGACACAGCG-----------------------ACACAAACG
GTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTCGGGTTAGG-GTTCGGG---TTCGGGTTCGGGTTCGGGTTCGGGTTCGGGTTAGGGTTAGGGTTAGGGTAAC-C-CTAACCCTA-ACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACC

It looks better - we have less small gaps in template, though they are still there.

In [138]:
# Greater penalty for opening
for read in reads:
    for x in pairwise2.align.localxs(allele, read.seq, -5, -1)[0]:
        print(x)
    print()

----CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG-----------------------ACACAAACG
CTCACCCTAACCCTCACCCTAACCCTCACCCGAACCCTCACCCGAACCCTCACCCGAACCCGAACCCTCACCCGAACCCTAACCCTCACCCGAACCCTAACCCGGCTAGGGCTGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTGGGGTAGGGGTAGGGGTAGGGGTAGGGGTAGGGGTGGGGGTAGGG--------------------
65.0
4
205

----CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG-----------------------ACACAAACG-----------------------------------------
GTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTC--GGGTTAGGGTTCGGGTTCGGGTTCGGGTTCGGGTTCGGGTTCGGGTTAGGGTTAGGGTTAGGGTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCAAACCCAAACCCAAACCCGACCCCAAACCCGAACCCGACCCCGAAACCGAACCC
61

That\`s nice, no splits in template. Next take a look at alignments and scores in case of similar sequences or absolutely different

In [207]:
# There are several variants which part of v segment read can contain - first part of core sequence,
# almost all core sequence, last part of core sequence plus 7-9-mers, both 7-9-mers or just nonamer.
prime_core = allele[:150]
core = allele[20:len(allele) - 59]
last_core = allele[80:len(allele) - 39]
core_hept = allele[80:len(allele) - 32]
core_conserved = allele[100:len(allele) - 32]
conserved_spacer = allele[len(allele) - 39:]

for part in (prime_core, core, last_core, core_hept, core_conserved, conserved_spacer):
    for x in pairwise2.align.localxs(allele, part, -5, -1)[0]:
        print(x)
    print()

CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG-----------------------ACACAAACG
CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGAT-----------------------------------------------------------------------
150.0
0
150

CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG-----------------------ACACAAACG
--------------------CCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGATGGCCAAGGGCAG-----------------------------------------------------------
142.0
20
162

CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTG

While sequences identical everything maps nice. Now cases with mismatches and read random ends

In [198]:
conserved_with_random_spacer = 'CACAGCG' + create_spacer(23) + 'ACACAAACG'
just_conserved = 'CACAGCG' + 'ACACAAACG'
for part in [conserved_with_random_spacer, just_conserved]:
    for x in pairwise2.align.localxs(allele, part, -5, -1)[0]:
        print(x)
    print()

CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG-----------------------ACACAAACG
-----------CACAGCGTACCTATTCTCAAACCTTTTGACACACAAACG---------------------------------------------------------------------------------------------------------------------------------------------------------------------------
18.0
11
50

CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG-----------------------ACACAAACG
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CACAGCGACACAAACG
9.0
212
221



It\`s not good - part with 7-9-mers and random spacer aligned not to it counterpart. Let\`s tweak match-mismatch scores.

In [211]:
# Same part with another score function
conserved_with_random_spacer = 'CACAGCG' + create_spacer(23) + 'ACACAAACG'
prime_part = create_spacer(70) + allele[:60]
mid_part = allele[3:115]
end_part = 'CTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG' + create_spacer(23)
really_end_part = 'GGCAGGGCCGCACATTCCCGTGGGACACAGCG' + create_spacer(23) + 'ACACAAACG' + create_spacer(20)
nonamer_part = create_spacer(23) + 'ACACAAACG' + create_spacer(80)
nonamer = 'ACACAAACG' + create_spacer(100)
for part in [conserved_with_random_spacer, prime_part, mid_part, end_part, really_end_part, nonamer_part, nonamer]:
    for x in pairwise2.align.localms(allele, part, 1, -2,  -5, -1)[0]:
        print(x)
    print()

# Another parts

CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG-----------------------ACACAAACG
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CACAGCGAGTGGAGCAAATATAACGAAGGAACACAAACG
9.0
212
221

----------------------------------------------------------------------CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG-----------------------ACACAAACG
ACATCATACTCTGGATCACGGGTACTCACCGACGACGATACGTTACCGGATCTGGACATCGTGAGCGAAACTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAG-------------------------------------------------------------------------------------------------------------------------

Good, now everything same except some noise due to errors

In [227]:
nucl = 'A'
for o in list(filter(lambda x: x != nucl, ['A', 'T', 'G', 'C']))[0]:
    print(o)
allele[:20]

'CTGGGCCTGGACCCAGCAGC'

In [229]:
# I`ve chosen error rate equal to 0.1 to play safe
def sequencing(fragment, error_rate):
    """Return given fragment with some errors"""
    return ''.join([nucl if random.random() > error_rate 
                    else list(filter(lambda x: x != nucl, ['A', 'T', 'G', 'C']))[0] 
                    for nucl in fragment])

conserved_with_random_spacer = sequencing('CACAGCG', 0.1) + create_spacer(23) + sequencing('ACACAAACG', 0.1)
prime_part = create_spacer(70) + sequencing(allele[:60], 0.1)
mid_part = sequencing(allele[3:115], 0.1)
end_part = sequencing('CTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG', error_rate=0.1) + create_spacer(23)
really_end_part = sequencing('GGCAGGGCCGCACATTCCCGTGGGACACAGCG', error_rate=0.1) + create_spacer(23) + sequencing('ACACAAACG', error_rate=0.1) + create_spacer(20)
nonamer_part = create_spacer(23) + sequencing('ACACAAACG', error_rate=0.1) + create_spacer(80)
nonamer = sequencing('ACACAAACG', error_rate=0.1) + create_spacer(100)
for part in [conserved_with_random_spacer, prime_part, mid_part, end_part, really_end_part, nonamer_part, nonamer]:
    for x in pairwise2.align.localms(allele, part, 1, -2,  -5, -1)[0]:
        print(x)
    print()

CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG-----------------------ACACAAACG
----------------------------------------------------------------------------------------------------------------------------------------CAAAGCGCCTTGTGCACTACTAAGCGCAGAACACATTCG----------------------------------------------
7.0
167
174

----------------------------------------------------------------------CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG-----------------------ACACAAACG
GGAGTATTGTTCTGCGAACCTTTATTCACCAAGACGGTCCAGCAGGGGTCTGCTCAAATCATCATACGCACTGGGCCTGGACAAAACAGCCCTCTGGGAAGGCGCTGGAGCACCTCAGCTCCAAGGGCAG-------------------------------------------------------------------------------------------------------------------------

Two fragments failed to align properly - both of them include just heptamer and/or nonamer, so these regions are not enough to align read if there are quite many substitutions in 7-9mers. Or sequenced read randomly similar to part of template.

In [230]:
# Same as previous cell, just lower error rate - 0.05
conserved_with_random_spacer = sequencing('CACAGCG', 0.05) + create_spacer(23) + sequencing('ACACAAACG', 0.05)
prime_part = create_spacer(70) + sequencing(allele[:60], 0.05)
mid_part = sequencing(allele[3:115], 0.05)
end_part = sequencing('CTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG', error_rate=0.05) + create_spacer(23)
really_end_part = sequencing('GGCAGGGCCGCACATTCCCGTGGGACACAGCG', error_rate=0.05) + create_spacer(23) + sequencing('ACACAAACG', error_rate=0.05) + create_spacer(20)
nonamer_part = create_spacer(23) + sequencing('ACACAAACG', error_rate=0.05) + create_spacer(80)
nonamer = sequencing('ACACAAACG', error_rate=0.05) + create_spacer(100)
for part in [conserved_with_random_spacer, prime_part, mid_part, end_part, really_end_part, nonamer_part, nonamer]:
    for x in pairwise2.align.localms(allele, part, 1, -2,  -5, -1)[0]:
        print(x)
    print()

CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG-----------------------ACACAAACG
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------CACAGCGTCCTCAAGGGTGAGAACACGTAGACACAAACG
9.0
212
221

----------------------------------------------------------------------CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG-----------------------ACACAAACG
GGTTTGGAGGGCCCTCGTGTGACAATGGAGCGTCTCCGGGCGCAAGAAGAAGAACCCGAACCAACTATGCCTGGGCCTGGACCCTGCAGCCCTCTAGGAAGGCGCTGGAGCACCTCAGCTCCAGGGGCAG-------------------------------------------------------------------------------------------------------------------------

1 fragment still unaligned but it is better. With lower error rate this approach shows better results. As you can see score of alignment is proportional to the size of aligned fragment (obviously) and it ranges from ~5 to ~100 in cases of nonamer and 100nt accordingly.

In [239]:
# Alignment of different sequence
no_similarity_equal = ''.join([random.choice(list(filter(lambda x: x != nucl, ['A', 'T', 'G', 'C']))) for nucl in allele])
no_similarity_shorter = ''.join([random.choice(list(filter(lambda x: x != nucl, ['A', 'T', 'G', 'C']))) 
                                 for nucl in range(200)])
no_similarity_longer = ''.join([random.choice(list(filter(lambda x: x != nucl, ['A', 'T', 'G', 'C']))) 
                                for nucl in range(300)])
for part in [no_similarity_equal, no_similarity_shorter, no_similarity_shorter]:
    for x in pairwise2.align.localms(allele, part, 1, -2,  -5, -1)[0]:
        print(x)
    print()

CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG-----------------------ACACAAACG---------------------------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------------------------TCAACTTACACGGGTCGTCTTAAGGTTACTTAAAACAAATACAGGGGAAAGGGATAAAGTGCGTTCTGCGGATGTGAGAAAAGGACATAGTGATCGTTCAAACGATCGAGTTATTGCAATGTGTGCGGGTGATTGCTGCGATAATAGCCAAATGTTTATATAATGTTGTTCACTATTACACGATGCTGAGTTTATAGCCCCCTCGACGCAACGTCTCCTTA
6.0
215
221

--------------------------CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCG-----------------------ACACAAACG
TATCCAGCTAGGAAGATTCAAATACGTGACGAATTCTGTAATTCCGCTCCCATGTGATGCT

Good news - score is low, bad - we can\`t distinguish by score read with nonamer from irrelevant read.

## Conclusion
1. pairwise2.align.localms function with following parameters
    - match = 1
    - mismatch = -2
    - gap opening = -5
    - gap extension = -1
   
   looks suitable for our task
2. It is possible to find reads with long stretches of template sequence by alignment score
3. It looks hard to differ reads with short stretches of template sequence (<20) by alignment score. Probably we should use another method for it

In [114]:
SeqIO.write(cores, '10cores', 'fasta')

10

In [117]:
from v_segment_generation import combinations


path_to_genes = '10cores'
path_to_heptamers='../data/conserve/hv7'
path_to_nonamers='../data/conserve/hv9'
length=23
monomers=('A', 'T', 'G', 'C')
cum_distribution=(0.25, 0.5, 0.75, 1)

vs = combinations(path_to_genes = '10cores',
path_to_heptamers='../data/conserve/hv7',
path_to_nonamers='../data/conserve/hv9',
length=23,
monomers=('A', 'T', 'G', 'C'),
cum_distribution=(0.25, 0.5, 0.75, 1))

In [126]:
print(pairwise2.align.localxx(vs[0], str(reads[0].seq))[0])

('CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGATGGCCAAGGGCAGGGCCGCACATTCCCGTGGGACACAGCGTCCCCTCGGCCCACCGGGGAACGACACAAACG', 'CTGGGCCTGGACCCAGCAGCCCTCTGGGAAGGCGCTGGGGCACCTCAGCTCCAGGGGCAGCACACACTTCAGCCCAGCCTTTCTGGGCCAACTCTCCATCTGTAGAGACACATCCAAGGCCCAGTTATCCCTGCAGCTGAGCTCCGTGATGGCCAAGGGCAGGGCCGCACA-T----T----C-C--CG-----T-----------GG---G------A--', 182.0, 0, 219)


In [89]:
f = '''CLUSTAL X (1.81) multiple sequence alignment


COATB_BPIKE/30-81                   AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIRLFKKFSS
Q9T0Q8_BPIKE/1-52                   AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIKLFKKFVS
COATB_BPI22/32-83                   DGTSTATSYATEAMNSLKTQATDLIDQTWPVVTSVAVAGLAIRLFKKFSS
COATB_BPM13/24-72                   AEGDDP---AKAAFNSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTS
COATB_BPZJ2/1-49                    AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFAS
Q9T0Q9_BPFD/1-49                    AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTS
COATB_BPIF1/22-73                   FAADDATSQAKAAFDSLTAQATEMSGYAWALVVLVVGATVGIKLFKKFVS

COATB_BPIKE/30-81                   KA
Q9T0Q8_BPIKE/1-52                   RA
COATB_BPI22/32-83                   KA
COATB_BPM13/24-72                   KA
COATB_BPZJ2/1-49                    KA
Q9T0Q9_BPFD/1-49                    KA
COATB_BPIF1/22-73                   RA'''

with open('f', 'w') as dest:
    dest.write(f)

In [92]:
for i in AlignIO.parse('f', 'clustal'):
    print(i)

SingleLetterAlphabet() alignment with 7 rows and 52 columns
AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIRL...SKA COATB_BPIKE/30-81
AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIKL...SRA Q9T0Q8_BPIKE/1-52
DGTSTATSYATEAMNSLKTQATDLIDQTWPVVTSVAVAGLAIRL...SKA COATB_BPI22/32-83
AEGDDP---AKAAFNSLQASATEYIGYAWAMVVVIVGATIGIKL...SKA COATB_BPM13/24-72
AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKL...SKA COATB_BPZJ2/1-49
AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKL...SKA Q9T0Q9_BPFD/1-49
FAADDATSQAKAAFDSLTAQATEMSGYAWALVVLVVGATVGIKL...SRA COATB_BPIF1/22-73


221

# STOCKHOLM 1.0
#=GF SQ 8
IGHV1-18*02 CAGGTTCAGCTGGTGCAGTCTGGAGCTGAGGTGAAGAAGCCTGGGGCCTCAGTGAAGGTCTCCTGCAAGGCTTCTGGTTACACCTTTACCAGCTATGGTATCAGCTGGGTGCGACAGGCCCCTGGACAAGGGCTTGAGTGGATGGGATGGATCAGCGCTTACAATGGTAACACAAACTATGCACAGAAGCTCCAGGGCAGAGTCACCATGACCACAGACACATCCACGAGCACAGCCTACATGGAGCTGAGGAGCCTAAGATCTGACGACACGGCC--------------------
#=GS IGHV1-18*02 AC IGHV1-18*02
#=GS IGHV1-18*02 DE IGHV1-18*02
IGHV1-18*03 CAGGTTCAGCTGGTGCAGTCTGGAGCTGAGGTGAAGAAGCCTGGGGCCTCAGTGAAGGTCTCCTGCAAGGCTTCTGGTTACACCTTTACCAGCTATGGTATCAGCTGGGTGCGACAGGCCCCTGGACAAGGGCTTGAGTGGATGGGATGGATCAGCGCTTACAATGGTAACACAAACTATGCACAGAAGCTCCAGGGCAGAGTCACCATGACCACAGACACATCCACGAGCACAGCCTACATGGAGCTGAGGAGCCTGAGATCTGACGACATGGCCGTGTATTACTGTGCGAGAGA
#=GS IGHV1-18*03 AC IGHV1-18*03
#=GS IGHV1-18*03 DE IGHV1-18*03
IGHV1-18*04 CAGGTTCAGCTGGTGCAGTCTGGAGCTGAGGTGAAGAAGCCTGGGGCCTCAGTGAAGGTCTCCTGCAAGGCTTCTGGTTACACCTTTACCAGCTACGGTATCAGCTGGGTGCGACAGGCCCCTGGACAAGGGCTTGAGTGGATGGGATGGATCAGCGCTTACAATGGTAACACAAACTATGCACAGAAGCTCCAGGGCAGAGTCACCATGACCACA

In [110]:
for i in pairwise2.align.localxx(allele.seq, reads[1])[0]:
    print(i, end='\n***\n')


In [7]:
fas = '''>L22582|IGHV1-69*01|Homo sapiens|F|V-REGION|376..671|296 nt|1| | | | |296+0=296| | |
caggtgcagctggtgcagtctggggctgaggtgaagaagcctgggtcctcggtgaaggtc
tcctgcaaggcttctggaggcaccttcagcagctatgctatcagctgggtgcgacaggcc
cctggacaagggcttgagtggatgggagggatcatccctatctttggtacagcaaactac
gcacagaagttccagggcagagtcacgattaccgcggacgaatccacgagcacagcctac
atggagctgagcagcctgagatctgaggacacggccgtgtattactgtgcgagaga
>Z27506|IGHV1-69*02|Homo sapiens|F|V-REGION|1..294|294 nt|1| | | | |294+0=294| | |
caggtccagctggtgcaatctggggctgaggtgaagaagcctgggtcctcggtgaaggtc
tcctgcaaggcttctggaggcaccttcagcagctatactatcagctgggtgcgacaggcc
cctggacaagggcttgagtggatgggaaggatcatccctatccttggtatagcaaactac
gcacagaagttccagggcagagtcacgattaccgcggacaaatccacgagcacagcctac
atggagctgagcagcctgagatctgaggacacggccgtgtattactgtgcgaga'''

x = pairwise2.align.globalxx('''caggtgcagctggtgcagtctggggctgaggtgaagaagcctgggtcctcggtgaaggtc
tcctgcaaggcttctggaggcaccttcagcagctatgctatcagctgggtgcgacaggcc
cctggacaagggcttgagtggatgggagggatcatccctatctttggtacagcaaactac
gcacagaagttccagggcagagtcacgattaccgcggacgaatccacgagcacagcctac
atggagctgagcagcctgagatctgaggacacggccgtgtattactgtgcgagaga'''.replace('\n', ''), 
'''caggtccagctggtgcaatctggggctgaggtgaagaagcctgggtcctcggtgaaggtc
tcctgcaaggcttctggaggcaccttcagcagctatactatcagctgggtgcgacaggcc
cctggacaagggcttgagtggatgggaaggatcatccctatccttggtatagcaaactac
gcacagaagttccagggcagagtcacgattaccgcggacaaatccacgagcacagcctac
atggagctgagcagcctgagatctgaggacacggccgtgtattactgtgcgaga'''.replace('\n', ''))
print(format_alignment(*x[0]))

caggtgc-agctggtgcag-tctggggctgaggtgaagaagcctgggtcctcggtgaaggtctcctgcaaggcttctggaggcaccttcagcagctatg-ctatcagctgggtgcgacaggcccctggacaagggcttgagtggatgggag-ggatcatccctatct-ttggtac-agcaaactacgcacagaagttccagggcagagtcacgattaccgcggacgaa-tccacgagcacagcctacatggagctgagcagcctgagatctgaggacacggccgtgtattactgtgcgagaga
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
caggt-ccagctggtgca-atctggggctgaggtgaagaagcctgggtcctcggtgaaggtctcctgcaaggcttctggaggcaccttcagcagctat-actatcagctgggtgcgacaggcccctggacaagggcttgagtggatggga-aggatcatccctatc-cttggta-tagcaaactacgcacagaagttccagggcagagtcacgattaccgcggac-aaatccacgagcacagcctacatggagctgagcagcctgagatctgaggacacggccgtgtattactgtgc--gaga
  Score=287

